In [1]:
from unsloth_mlx import FastLanguageModel, SFTTrainer, SFTConfig
from datasets import load_dataset

# Load any HuggingFace model (1B model for quick start)
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="mlx-community/Llama-3.2-1B-Instruct-4bit",
    max_seq_length=2048,
    load_in_4bit=True,
)

# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha=16,
)

# Load a dataset (or create your own)
dataset = load_dataset("yahma/alpaca-cleaned", split="train[:100]")

# Train with SFTTrainer (same API as TRL!)
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    tokenizer=tokenizer,
    args=SFTConfig(
        output_dir="outputs",
        per_device_train_batch_size=2,
        learning_rate=2e-4,
        max_steps=50,
    ),
)
trainer.train()

# Save (same API as Unsloth!)
model.save_pretrained("lora_model")  # Adapters only
model.save_pretrained_merged("merged", tokenizer)  # Full model
model.save_pretrained_gguf("model", tokenizer, quantization_method="q4_k_m")  # GGUF


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

LoRA configuration set: rank=16, alpha=16, modules=['q_proj', 'k_proj', 'v_proj', 'o_proj'], dropout=0.0
Trainer initialized:
  Output dir: outputs
  Adapter path: adapters
  Learning rate: 0.0002
  Iterations: 50
  Batch size: 2
  LoRA r=16, alpha=16
  Native training: True
  LR scheduler: cosine
  Grad checkpoint: False
Starting Fine-Tuning

[Using Native MLX Training]

Applying LoRA adapters...
Applying LoRA to 16 layers: {'rank': 16, 'scale': 1.0, 'dropout': 0.0, 'keys': ['self_attn.q_proj', 'self_attn.k_proj', 'self_attn.v_proj', 'self_attn.o_proj']}
✓ LoRA applied successfully to 16 layers
  Trainable LoRA parameters: 128
Preparing training data...
  Detected format: alpaca
✓ Prepared 100 training samples
  Saved to: outputs/train.jsonl
✓ Created validation set (copied from train)

Training configuration:
  Iterations: 50
  Batch size: 2
  Learning rate: 0.0002
  LR scheduler: cosine
  Grad checkpoint: True
  Adapter file: adapters/adapters.safetensors

Loaded 100 training sample

Calculating loss...: 100%|██████████| 25/25 [00:04<00:00,  5.82it/s]

Iter 1: Val loss 1.988, Val took 4.302s


Iter 10: Train loss 1.485, Learning Rate 1.844e-04, It/sec 1.362, Tokens/sec 607.188, Trained Tokens 4457, Peak mem 2.273 GB
Iter 20: Train loss 1.327, Learning Rate 1.368e-04, It/sec 3.126, Tokens/sec 811.520, Trained Tokens 7053, Peak mem 2.273 GB
Iter 30: Train loss 1.276, Learning Rate 7.513e-05, It/sec 2.256, Tokens/sec 858.579, Trained Tokens 10858, Peak mem 2.273 GB
Iter 40: Train loss 1.366, Learning Rate 2.295e-05, It/sec 2.489, Tokens/sec 799.567, Trained Tokens 14070, Peak mem 2.273 GB


...

Iter 50: Val loss 1.179, Val took 3.860s


Iter 50: Train loss 1.273, Learning Rate 1.973e-07, It/sec 1.904, Tokens/sec 720.759, Trained Tokens 17856, Peak mem 2.273 GB
Saved final weights to adapters/adapters.safetensors.

Training Complete!
  Adapters saved to: adapters
Saving LoRA adapters to lora_model...
✓ Adapters saved to lora_model
Saving merged model to merged...
Saving model to merged...
✓ Model saved to merged
  LoRA adapters will be fused from: adapters
Exporting to GGUF format...
Exporting model to GGUF format...
  Model: mlx-community/Llama-3.2-1B-Instruct-4bit
  Output: model/model.gguf
  Adapters: adapters

Running: mlx_lm.fuse --model mlx-community/Llama-3.2-1B-Instruct-4bit --export-gguf --gguf-path model/model.gguf --adapter-path adapters
Error during GGUF export: 
Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 98304.00it/s]
Traceback (most recent call last):
  File "/Users/mizuy/lab/langmodel/.venv/bin/mlx_lm.fuse", line 10, in <module>
    sys.exit(main())
             ~~~~^^
  File "/Users/mizuy/lab/

usage: mlx_lm.convert [-h] [--hf-path HF_PATH] [--mlx-path MLX_PATH] [-q]
                      [--q-group-size Q_GROUP_SIZE] [--q-bits Q_BITS]
                      [--q-mode {affine,mxfp4}]
                      [--quant-predicate {mixed_2_6,mixed_3_4,mixed_3_6,mixed_4_6}]
                      [--dtype {float16,bfloat16,float32}]
                      [--upload-repo UPLOAD_REPO] [-d] [--trust-remote-code]
mlx_lm.convert: error: unrecognized arguments: --export-gguf


Alternative method also failed: Command '['mlx_lm.convert', '--hf-path', 'mlx-community/Llama-3.2-1B-Instruct-4bit', '-q', '--export-gguf']' returned non-zero exit status 2.

Manual export command:
  mlx_lm.fuse --model mlx-community/Llama-3.2-1B-Instruct-4bit --export-gguf --gguf-path model/model.gguf


CalledProcessError: Command '['mlx_lm.convert', '--hf-path', 'mlx-community/Llama-3.2-1B-Instruct-4bit', '-q', '--export-gguf']' returned non-zero exit status 2.

In [2]:
dataset

Dataset({
    features: ['output', 'input', 'instruction'],
    num_rows: 100
})

In [3]:
model